# Webscraping Blue Apron
   * Required multiple tools
   * Required a lot of trial and error
   * Several attempts at webscraping the right information
   * main tools used were pandas, beautifulsoup, selenium webdriver, pickle

In [160]:
import numpy as np
import pandas as pd
import requests
from IPython.core.display import HTML
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import pickle
import matplotlib.pyplot as plt
import psycopg2
import sys
%matplotlib inline
from sqlalchemy import create_engine
driver = webdriver.PhantomJS(executable_path='/Users/Guillermo/anaconda/bin/phantomjs')

# Testing samples for webscrape of Blue Apron

In [ ]:

ad1 = 'https://www.blueapron.com/cookbook'
ad3 = 'https://www.blueapron.com/recipes/sweet-potato-quesadillas-with-green-chiles-arugula-avocado-salad'
ad2 = 'https://www.blueapron.com/recipes/za-atar-spiced-chicken-souvlaki-with-tahini-yogurt-sauce-roasted-potatoes'
testlist = [ad2,ad3]

# Scrape Blue Apron using phatomJS then turn page source into a beaultiful object

In [ ]:

driver = webdriver.PhantomJS(executable_path='/Users/Guillermo/anaconda/bin/phantomjs')
driver.set_window_size(1024, 768) 
driver.get(ad3)
page_html2 = driver.page_source
#HTML(driver.page_source)
soup2 = BeautifulSoup(page_html2, "lxml")

# Creates original websacrape from Blue Apron's Cookbook page
* Ran into trouble with loading the entire page
* Problem resolved using "driver.execute script" below, it keeps scrolling down each time it is instantiated
* Need to run this 30 times to get all of the recipes
* Checked length of recipes for sanity check
* This was a Brute force method but it was effective at gathering all the data.

In [ ]:

for i in range(0,30):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
page_html4 = driver.page_source

soupblue2 = BeautifulSoup(page_html4, "lxml")

## Pulls the recipe url for each of the recipes in the CookBook

In [ ]:
# pulls the recipe url for each of the recipes in the CookBook
recipeurl =[]
for link in soupblue2.findAll('div', class_="recipe-thumb col-md-4"):
    #print(link.text)
    for lp in link.findAll('a'):
        recipeurl.append(lp['href'])
        #print lp['href']
        #print '\n'

## Pull the recipe name for each of the recipes in the cookbook
* Appends the names to a list

In [ ]:
# pull the recipe name for each of the recipes in the cookbook
# appends the names to a list
i = 0
recipename = [ ]
for d in soupblue2.findAll('div', class_='recipe-img-link'):
     for g in d.findAll(class_= "img-flex"):
            # for p in d.findAll('img):
            #print(i, g)
            #print(i, g['alt'])
            
            recipename.append(g['alt'])
            #print(g['src'])
            #print('\n')
            i += 1

In [ ]:
# Concatenates the home URL with the urls from each recipe to be able to then call each link
urlrecp = "https://www.blueapron.com"
recipeurl2 = [ urlrecp + i for i in recipeurl]

# zips recipe names and recipe urls into one list then conversts it into a Panda Data Frame
newrecipelist = zip(recipename,recipeurl2)
myrecipes3 = pd.DataFrame(newrecipelist)

# rename column of panda dataframe
myrecipes3.columns(1="name")
myrecipes3=myrecipes3.rename(columns = {1:'URL'})

## Create engine to store data in Postgres SQl
* requires SQLalchemy

In [20]:
# create engine to store data in Postgres SQl
# requires SQLalchemy

engine = create_engine('postgresql://Guillermo@localhost:5432/mydb')
! pg_ctl start -D ~/psql

pg_ctl: another server might be running; trying to start server anyway
server starting


In [173]:
#sanity check using psycog to open and check connection to database

#define connection string specifing the server, the database, the user and the password
constr =  "host='localhost' dbname='mydb' user='Guillermo' password = 'DB1076pp'"

#Use the connection string to establish a connection to the server. The return type is of type SQLConnection
conn = psycopg2.connect(constr)

#Create a Curser Instance using the connection
cursor = conn.cursor()

#Connection Status
conn.status

1

# Establish setting up Postgres SQL Database

In [ ]:
# Store Panda dataframe as SQL table
myrecipes3.to_sql("BlueApronRecipes", engine)

In [162]:
# Pull Table of recipes into a Panda dataframe
sql = 'SELECT * FROM "BlueApronRecipes"'
myrecp = pd.read_sql(sql, engine)

print len(myrecp)  # Sanity Check

# iterate through panda data frame to create list of recipe urls
myrecp2 = [ data[3] for data in myrecp.itertuples()]
myrecp3 = [ data[2] for data in myrecp.itertuples()]

930


# First attempt at websscraping all Recipes

In [ ]:
# Webscrape of recipes from Blue Apron. 
# second try to work out kinks in data pull 
start = time.time()
#ingredients = []
print "go1"  # Sanity check
#newrecpe = []
driver = webdriver.PhantomJS(executable_path='/Users/Guillermo/anaconda/bin/phantomjs')
driver.set_window_size(1024, 768) 
print "go"  # Sanity Check
#for i in range(0, len(testlist)):
for i in range(619, len(myrecp2)):
    #driver.get(testlist[i])
    time.sleep(1)  # try to time it so Blue Apron doesn't kick me out
    driver.get(myrecp2[i]) # use precreated list of recipes from previous scrape to pull each individual recipe
    print myrecp2[i]
    print i
    page_html2 = driver.page_source
    soup2 = BeautifulSoup(page_html2, "lxml")
    newrecpe.append(soup2)  # list of beaultiful soup obeject for each recipe
    for o in soup2.findAll('ul', class_='ingredients-list'):
        ingredients.append((str(o.text.encode('utf-8')).replace('\n\n\n\n\n',',').replace('\n','').replace('\xc2\xbe','.75').replace("'",''), myrecp2[i]))
        #ingredients = ingredients[0].split(',')
        #print(str(o.text.encode('utf-8')))
        #print('\n')
        #print ingredients

end = time.time()                           
print "Completed in {0} minutes".format((end.start)/60)


# Start using pickle to store data easier for HTML, XML code

In [ ]:
# pickeling file of all beautifulsoup obejcts representing all of the recipes
# allows me to store the object and then retriev it for additional processing without having to rescrape 

sys.setrecursionlimit(3000) # needed to import sys and extend the recursion limit because the object was so large

# I should have save each as page.source not as a beautiful soup obeject.
# decided not to redo it due to time constraints and not willing to risk being blocked by Blue Apron

with open('recpesoup.pkl', 'w') as picklefile:
    pickle.dump(newrecpe, picklefile)

In [161]:
# Reloads pickle file into list

with open('recpesoup.pkl', 'r') as picklefile:
    myrecpld = pickle.load(picklefile)

In [ ]:
# Need to webscrape the comments from each of the recipes
# This required scraping the beautiful soup object for the iframe 'facebook' url to pull in comments
# Blue Apron only displays a certain number of comments at a time, I needed all of them
# I scraped the total count of the 

newrecplist = []
commentlist = []
for i in range(0,929):
    soup6 = myrecpld[i]
    for o in soup6.findAll('iframe', class_="fb_ltr fb_iframe_widget_lift"):
        ifrmUrl = o['src']
        print i 
        print ifrmUrl
        time.sleep(1)
        
        gg = ifrmUrl.split('numposts')
        
        driver.get(ifrmUrl)
        page_html4 = driver.page_source
        

        soup4 = BeautifulSoup(page_html4, "lxml")
        
        for c in soup4.findAll('span', class_=" _50f7"): # scrape for comment count
            #print c.text
            rr = c.text.encode('ascii', 'ignore').split(' ')[0]
            if int(rr) < 1:
                print "Url {0} doesn't have any comments".format(ifrmUrl)
            jjj =gg[1].find('&')
            ff = gg[1][jjj:]
            newUrl =gg[0]+'numposts='+rr+ff 
            driver.get(newUrl)
            page_html5 = driver.page_source
            commentlist.append(page_html5)
            newrecplist.append([myrecp2[i],ifrmUrl,c.text,rr])

In [260]:
#print len(commentlist)
print len(newrecplist)
print newrecplist[1]
#print commentlist[0] # Sanity Check
print len(myrecpld)

74
https://www.facebook.com/plugins/comments.php?api_key=268433919926057&channel_url=https%3A%2F%2Fstaticxx.facebook.com%2Fconnect%2Fxd_arbiter%2Fr%2FYGoENyUbMBG.js%3Fversion%3D42%23cb%3Df39d273c3c%26domain%3Dwww.blueapron.com%26origin%3Dhttps%253A%252F%252Fwww.blueapron.com%252Ffa944c34c%26relation%3Dparent.parent&href=http%3A%2F%2Fwww.blueapron.com%2Frecipes%2Fpaprika-spiced-catfish-with-cabbage-slaw-roasted-purple-potato-wedges&locale=en_US&numposts=3&sdk=joey&version=v2.7&width=100%25
929


In [ ]:
with open('reccomments.pkl', 'w') as picklefile:
    pickle.dump(commentlist, picklefile)

In [11]:
with open('allcommentlist.pkl', 'w') as picklefile:
    pickle.dump(commtrecplist, picklefile)

In [141]:
# not to be used it is an old list
with open('reccomlist2.pkl', 'r') as picklefile:
    commentlist4 = pickle.load(picklefile)

In [313]:
#commentlist = commentlist3
#print len(commentlist4)
print len(commentlist3)
commentlist[1]

852


u'<!DOCTYPE html><html lang="en" id="facebook" class="svg "><head><meta charset="utf-8"><meta name="referrer" content="default" id="meta_referrer"><script>function envFlush(a){function b(c){for(var d in a)c[d]=a[d];}if(window.requireLazy){window.requireLazy([\'Env\'],b);}else{window.Env=window.Env||{};b(window.Env);}}envFlush({"ajaxpipe_token":"AXivYV4Sh3nHzL1C","no_cookies":1,"khsh":"0`sj`e`rm`s-0fdu^gshdoer-0gc^eurf-3gc^eurf;1;enbtldou;fduDmdldourCxO`ld-2YLMIuuqSdptdru;qsnunuxqd;rdoe-0unjdojnx-0unjdojnx0-0gdubi^rdbsduOdv-0`sj`e`r-0q`xm`r-0StoRbs`qhof-0mhoj^q`xm`r"});</script><style></style><script>__DEV__=0;CavalryLogger=false;(function(a){var b=a.Env,c=a.performance;if(!c||!c.timing||!Object.getOwnPropertyDescriptor||!Object.defineProperty||!(b.timeslice_categories&&b.timeslice_categories.reflow))return;var d=c.timing.navigationStart,e=location.search.indexOf(\'detectreflows\')>-1,f=5;function g(){return c.now()+d;}function h(){}function i(n,o,p){return function q(){var r=void 0,s=0

In [163]:
myrecp2[1]

u'https://www.blueapron.com/recipes/sweet-potato-quesadillas-with-green-chiles-arugula-avocado-salad'

In [66]:
text3 = soup2.find(class_="nutrition-information row")

text4 = text3.find(class_="col-xs-4")
text7 = text3.find(class_="col-xs-5")
text5 = text4.find('span')
text6 = text7.find(itemprop="calories")
#text6 = text7.find(class_="data-calories")


print text4.text.split('Cook Time')[1]
print text6.text


35-45mins

550


In [199]:
#rerun with recipes that were not pulled
recpPlusIngredient3 = []
notfoundlist2 = []
for i in range(0, len(newrecpe)):
    soup2 = BeautifulSoup(newrecpe[i], "lxml")
    ingredients = []
    for o in soup2.findAll('ul', class_='ingredients-list'):
        #for k in o.findAll('div', class_='non-story'): #class_='amount' ):
             #for l in k.findAll('span', class_='amount'):
            ingredients.append(str(o.text.encode('utf-8')).replace('\n\n\n\n\n',',')) #.replace('\n','').replace('\xc2\xbe','.75').replace("'",''))
            #ingredients = ingredients[0].split(',')
            #print(str(o.text.encode('utf-8')))
            #print('\n')
            #print ingredients
    recpPlusIngredient3.append([newrecpe[i], ingredients]) #, cooktime, calories])

    #recpPlusIngredient3.append([newrecp[i], ingredients]) #, cooktime, calories])
print "done"    

done


# Webscrape recipes not fully pulled first time around

In [ ]:
#recpPlusIngredient3 = []
notfoundlist2 = []
for i in range(891, len(myrecpld)):
    soup2 = myrecpld[i]
    #print soup2.prettify()
    ingredients = []
    print i
    o = soup2.find('ul', class_='ingredients-list')
    try:
        for p in o.findAll(itemprop='ingredients'):
            ingredients.append(p.text)
    except AttributeError:
        pass
    try:
        rcpnm = soup2.find('div', class_="fb-like")
        rcpm2 = str(rcpnm).split('data-href=')[1]
        rcpm3 = rcpm2.split(' data-layout')[0]
    except AttributeError or IndexError:
        rcpnm = soup2.find('div', class_="fb-like fb_iframe_widget")
        rcpm2 = str(rcpnm).split('data-href=')[1]
        rcpm3 = rcpm2.split(' data-layout')[0]
   
    #print rcpm3
    recpPlusIngredient3.append([rcpm3, ingredients]) #, cooktime, calories])
print "done"

In [72]:
with open('recpPlusIngredient.pkl', 'w') as picklefile:
    pickle.dump(recpPlusIngredient, picklefile)

In [ ]:
# to list
myrecp2 = [ data[3] for data in myrecp.itertuples()]

In [69]:
print myrecp2[0]

https://www.blueapron.com/recipes/spicy-korean-rice-cakes-with-yu-choy-soft-boiled-eggs


In [ ]:
recipeurl2[929]

# First trial at pulling ingredients

In [ ]:
for i in range(0, len(myrecp))
ingredients = []
for o in soupblue3.findAll('ul', class_='ingredients-list'):
    #for k in o.findAll('div', class_='non-story'): #class_='amount' ):
         #for l in k.findAll('span', class_='amount'):
        ingre.append(str(o.text.encode('utf-8')).replace('\n\n\n\n\n',',').replace('\n','').replace('\xc2\xbe','.75').replace("'",''))
        ingredients = ingre[0].split(',')
        print(str(o.text.encode('utf-8')))
        print('\n')
        print ingredients

In [ ]:
for i in range(0, len(myrecp))
ingredients = []
soupblue3.findAll('ul', class_='ingredients-list'):
    #for k in o.findAll('div', class_='non-story'): #class_='amount' ):
         #for l in k.findAll('span', class_='amount'):
        ingre.append(str(o.text.encode('utf-8')).replace('\n\n\n\n\n',',').replace('\n','').replace('\xc2\xbe','.75').replace("'",''))
        ingredients = ingre[0].split(',')
        print(str(o.text.encode('utf-8')))
        print('\n')
        print ingredients

In [ ]:
len(ingredients)

In [154]:

soup5 = BeautifulSoup(commentlist[1], "lxml")


# Set up Functions to pull all recipes and neccesary information

In [319]:
# Functions to process comments
def get_name(sp):
    c = sp.find('noscript')
    try:    
        p = c.text
        gg = p.split('recipes')
        f = gg[1].find('F') + 1
        g = gg[1].find('&')
        return gg[1][f:g]
    except AttributeError:
        return "Not Found"

def numLikesPrcs(numLikes):  
    if len(numLikes) == 7:
        likes = numLikes[4]  # since it is a list the 4th and 6th elments of the list have the information that is neededdatepost = numLikes[6] # might need to check the lenght of this list to make sure its alwasy pulling the same information
        datepost = numLikes[6]
    elif len(numLikes) == 5:
        datepost = numLikes[4]
        likes = 0
    else:
        pass
        
    return likes, datepost
    
def createnumlike(x):
    numLikes = []
    for y in x.children:  
        try:
            numLikes.append(y.text)
        except AttributeError:
            pass

    return numLikes

def get_replies(sp):
    replyList = []
    totallikes = 0
    for x in sp.findAll('div', class_="_3-8y clearfix"):
        
        replNm = x.find(class_=" UFICommentActorName")
        replAtrbt = x.find('div', class_="_4q1v")
        replPost = x.find('div', class_="_30o4")
        jorg = x.find(class_="_2vq9 fsm fwn fcg")
      
        if replAtrbt == None :
            replAtrbt = "none"
        else: 
            replAtrbt = replAtrbt.text
        likes, postdate = numLikesPrcs(createnumlike(jorg))
            
        replyList.append([replNm.text, replAtrbt, replPost.text, likes, postdate])
        totallikes = totallikes + int(likes)
    return replyList, len(replyList), (totallikes+len(replyList))

def get_commentators(sp):
    recpname = get_name(sp)
    commttList = []
    totallikes = 0
    for x in sp.findAll('div', class_="_3-8y _5nz1 clearfix"):
        
        cmttrNm = x.find(class_=" UFICommentActorName")
        cmttrAtrbt = x.find('div', class_="_4q1v")
        cmttrPost = x.find('div', class_="_30o4")
        jorg = x.find(class_="_2vq9 fsm fwn fcg")
      
        if cmttrAtrbt == None :
            cmttrAtrbt = "none"
        else: 
            cmttrAtrbt = cmttrAtrbt.text
        likes, postdate = numLikesPrcs(createnumlike(jorg))
        
        replylist, numrply, numeng = get_replies(x)

        commttList.append([recpname,cmttrNm.text, cmttrAtrbt, cmttrPost.text, likes, postdate, numrply, replylist])
        totallikes = totallikes + int(likes) + numeng
    return commttList, len(commttList), recpname, totallikes 


In [321]:
print len(commtrecplist3)

496


In [323]:
commtrecplist3 = []
for i in range(0, len(newcommentlist)):
    soup1 =  BeautifulSoup(newcommentlist[i], "lxml")
    comments, numcommts, recpname, numeng = get_commentators(soup1)
    commtrecplist3.append([recpname, comments, numcommts, numeng])
    #print i
print len(commtrecplist3)

926


In [2]:
with open('reccomments.pkl', 'r') as picklefile:
    commentlist3 = pickle.load(picklefile)

In [75]:
# create list of all items that had comments pulled
pulledlist = [commtrecplist[i][0] for i in range(0, len(commtrecplist))]

In [78]:
#test pulling recipe name to comapre with pulled comment list
recpPlusIngredient[1][0].split('recipes/')[1] for i in range(0, len(commtrecplist) for p in recpPlusIngredient)

u'sweet-potato-quesadillas-with-green-chiles-arugula-avocado-salad'

In [97]:
#create list of recipes that have not had comments pulled
notpulled =[recpPlusIngredient[i][0] for i in range(0, len(recpPlusIngredient)) if recpPlusIngredient[i][0].split('recipes/')[1] not in pulledlist] 

In [98]:
# number of recipes not pulled
len(notpulled)

79

In [104]:
notpulled2 = [notpulled[i].replace("'", '') for i in range(0, len(notpulled))]
notpulled2[1]

u'https://www.blueapron.com/recipes/paprika-spiced-catfish-with-cabbage-slaw-roasted-purple-potato-wedges'

In [129]:
# run the recipe pull a second time
print notpulled[3]

https://www.blueapron.com/recipes/fresh-mozzarella-basil-pizza-with-alcosa-cabbage-salad


# Creates a list with comments, iframe, URL, Text and number of comments

In [144]:

for i in range(16,len(notpulled2)):
    driver.get(notpulled2[i])
    page_html2 = driver.page_source
    soup2 = BeautifulSoup(page_html2, "lxml")
    for o in soup2.findAll('iframe', class_="fb_ltr fb_iframe_widget_lift"):
        ifrmUrl = o['src']
 
        #print ifrmUrl
        time.sleep(1)
        
        gg = ifrmUrl.split('numposts')
        
        driver.get(ifrmUrl)
        page_html4 = driver.page_source
        

        soup4 = BeautifulSoup(page_html4, "lxml")
        
        for c in soup4.findAll('span', class_=" _50f7"): # scrape for comment count
            #print c.text
            rr = c.text.encode('ascii', 'ignore').split(' ')[0]
            print i, rr, notpulled2[i]
            if int(rr) < 1:
                print "Url {0} doesn't have any comments".format(ifrmUrl)
            jjj =gg[1].find('&')
            ff = gg[1][jjj:]
            newUrl =gg[0]+'numposts='+rr+ff 
            driver.get(newUrl)
            page_html5 = driver.page_source
            commentlist.append(page_html5)
            newrecplist.append([notpulled2[i],ifrmUrl,c.text,rr])

17 37 https://www.blueapron.com/recipes/basque-style-lamb-beef-piperade-with-tinkerbell-peppers-baby-leeks-garlic-rice
18 40 https://www.blueapron.com/recipes/spicy-pepperonata-pasta-with-tinkerbell-peppers-cherry-tomatoes-pine-nuts
19 158 https://www.blueapron.com/recipes/chicken-buttermilk-biscuit-sandwiches-with-shredded-collard-green-carrot-slaw
20 22 https://www.blueapron.com/recipes/southwestern-cheeseburgers-with-guacamole-warm-corn-tomato-salad
21 25 https://www.blueapron.com/recipes/spiced-pork-tacos-with-avocado-pickled-onion-elote-style-corn
22 20 https://www.blueapron.com/recipes/vegetable-pad-thai-with-tinkerbell-peppers-peanuts-thai-basil
23 60 https://www.blueapron.com/recipes/sweet-corn-tinkerbell-pepper-pizza-with-fontina-cheese-baby-kale
24 37 https://www.blueapron.com/recipes/green-chili-chicken-roll-with-lime-jicama-fries
25 10 https://www.blueapron.com/recipes/crispy-catfish-potato-salad-with-marinated-tomato-cucumber-onion
26 3 https://www.blueapron.com/recipes/zu

In [ ]:
commtrecplist2 = []
for i in range(0, len(commentlist)):
    soup1 =  BeautifulSoup(commentlist[i], "lxml")
    comments, numcommts, recpname = get_commentators(soup1)
    commtrecplist2.append([recpname, comments, numcommts])
    print i
print len(commtrecplist2)

In [ ]:
commtrecplist[1][1][1]

In [153]:
nlis = [newrecplist[i][0] for i in range(0, len(newrecplist))]
print len(nlis)

74


In [154]:
notpulled3 = [notpulled2[i] for i in range(0, len(notpulled2)) if notpulled2[i] not in nlis]

In [157]:
print notpulled3[1]

https://www.blueapron.com/recipes/seared-salmon-miso-soba-noodle-salad-with-fairy-tale-eggplant-baby-greens


In [158]:
with open('commentlist2filled.pkl', 'w') as picklefile:
    pickle.dump(commentlist, picklefile)

In [177]:
len(recpPlusIngredient)

929

In [314]:
newcommentlist = commentlist + commentlist3

In [315]:
with open('fullrawiframelist.pkl', 'w') as picklefile:
    pickle.dump(newcommentlist, picklefile)

In [324]:
commtrecplist3
with open('commentlistwithenagement.pkl', 'w') as picklefile:
    pickle.dump(commtrecplist3, picklefile)

## Webscrape of recipes from Blue Apron. 
* second try to work out kinks in data pull 

In [ ]:
# Webscrape of recipes from Blue Apron. 
# second try to work out kinks in data pull 
start = time.time()
#ingredients = []
print "go1"  # Sanity check
newrecpe = []
driver = webdriver.PhantomJS(executable_path='/Users/Guillermo/anaconda/bin/phantomjs')
driver.set_window_size(1024, 768) 
print "go"  # Sanity Check
#for i in range(0, len(testlist)):
for i in range(0, len(notfoundlist)):
    #driver.get(testlist[i])
    time.sleep(1)  # try to time it so Blue Apron doesn't kick me out
    driver.get(notfoundlist[i]) # use precreated list of recipes from previous scrape to pull each individual recipe
    print notfoundlist[i]
    print i
    page_html2 = driver.page_source
    soup2 = BeautifulSoup(page_html2, "lxml")
    newrecpe.append(page_html2)  # list of beaultiful soup obeject for each recipe
    #for o in soup2.findAll('ul', class_='ingredients-list'):
        #ingredients.append((str(o.text.encode('utf-8')).replace('\n\n\n\n\n',',').replace('\n','').replace('\xc2\xbe','.75').replace("'",''), myrecp2[i]))
        #ingredients = ingredients[0].split(',')
        #print(str(o.text.encode('utf-8')))
        #print('\n')
        #print ingredients

end = time.time()                           
print "Completed in {0} minutes".format((end - start)/60)

In [190]:
recpingretotal = [recpPlusIngredient[i] for i in range(0, len(recpPlusIngredient)) if recpPlusIngredient[i][0] not in notfoundlist]

In [182]:
len(recpingretotal)

891

In [192]:
fullypulledlist = recpingretotal + recpPlusIngredient2
with open('fullingredientlist.pkl', 'w') as picklefile:
    pickle.dump(fullypulledlist, picklefile)

In [300]:
with open('listrecpIngr_24.pkl', 'w') as picklefile:
    pickle.dump(recpPlusIngredient3, picklefile)